# Common

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 6]
plt.style.use('ggplot')

In [3]:
import gc

In [4]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from fastprogress import progress_bar

def parallel(func, job_list, n_jobs=12):
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        futures = [pool.submit(func, job) for job in job_list]
        for f in progress_bar(as_completed(futures), total=len(job_list)):
            pass
    return [f.result() for f in futures]

In [5]:
from warnings import filterwarnings

In [6]:
filterwarnings("ignore")

In [7]:
import datetime as dt

In [8]:
def to_dt(x):
    return dt.datetime.fromtimestamp(int(x/1000))

In [9]:
from multiprocessing import Pool

In [10]:
num_partitions = 100
num_cores = 12
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Agg Sessions

In [130]:
sessions = pd.read_pickle('../session.pkl')

In [49]:
sessions.shape

(5165760, 21)

In [52]:
sessions.dtypes

session_id                          category
start_timestamp                        int64
timezone                            category
timezone_offset                     category
previous_sessions_duration             int64
user_created_timestamp                 int64
is_user_first_session                   bool
country                             category
region                              category
city                                category
latitude                             float64
longitude                            float64
locale                              category
os_name                             category
session_index                          int64
device_id                           category
user_id_hash                        category
user_time                     datetime64[ns]
system_time                   datetime64[ns]
week_start                    datetime64[ns]
user_days                           category
dtype: object

In [51]:
sessions.sort_values(['system_time', 'user_id_hash'], inplace=True)

In [9]:
# sessions.set_index(['week_start', 'user_id_hash'], inplace=True)

In [131]:
sessions = sessions.loc[:, ['user_created_timestamp', 'user_id_hash']]

In [57]:
sessions.drop_duplicates(inplace=True)

In [58]:
sessions.shape

(619779, 2)

In [132]:
sessions.reset_index(inplace=True, drop=True)

In [133]:
sessions.user_created_timestamp = sessions.user_created_timestamp.apply(to_dt)

In [134]:
sessions.head()

,user_created_timestamp,user_id_hash
0,2018-10-07 01:04:49,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2018-10-07 01:04:49,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,2018-10-07 01:04:49,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2018-10-07 01:04:49,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2018-10-07 01:04:49,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [135]:
sessions.user_created_timestamp = sessions.user_created_timestamp.dt.date

In [12]:
def weekly_aggregate(df, t='week_start'):
    grp_cols = [t, 'user_id_hash']
    grpby_obj = df.groupby(grp_cols)
    # number of sessions
    temp = grpby_obj.city.count().to_frame().reset_index().\
                rename(columns={'city': 'no_city'})
    # median city
    # temp1 = grpby_obj.city.agg(lambda x: x.value_counts().index[0]
    #                            ).to_frame().reset_index()
    # last city
    temp1 = grpby_obj.city.last().to_frame().reset_index()
    
    # is user's first week
    temp2 = grpby_obj.is_user_first_session.max().to_frame().reset_index()

    # median country
    # temp3 = grpby_obj.country.agg(lambda x: x.value_counts().index[0]
    #                               ).to_frame().reset_index()
    temp3 = grpby_obj.country.last().to_frame().reset_index()

    # no of country
    temp4 = grpby_obj.country.agg(lambda x: x.unique().shape[0]
                                  ).to_frame().reset_index().\
                        rename(columns={'country': 'country_count'})
    # median region
    # temp5 = grpby_obj.region.agg(lambda x: x.value_counts().index[0]
    #                              ).to_frame().reset_index()
    temp5 = grpby_obj.region.last().to_frame().reset_index()

    # no of region
    temp6 = grpby_obj.region.agg(lambda x: x.unique().shape[0]
                                  ).to_frame().reset_index().\
                        rename(columns={'region': 'region_count'})
    # median locale
    # temp7 = grpby_obj.locale.agg(lambda x: x.value_counts().index[0]
    #                              ).to_frame().reset_index()
    temp7 = grpby_obj.locale.last().to_frame().reset_index()

    # median os_name
    # temp8 = grpby_obj.os_name.agg(lambda x: x.value_counts().index[0]
    #                               ).to_frame().reset_index()
    temp8 = grpby_obj.os_name.last().to_frame().reset_index()

    # no of days active
    temp9 = grpby_obj.user_days.agg(lambda x: x.unique().shape[0]).\
                to_frame().reset_index()
    # agg session time
    temp10 = grpby_obj.previous_sessions_duration.sum().\
                    to_frame().reset_index().\
                    rename(columns={'previous_sessions_duration': 'total_session_time'})
    # mean session time
    temp11 = grpby_obj.previous_sessions_duration.mean().\
                to_frame().reset_index().\
                rename(columns={'previous_sessions_duration': 'mean_session_time'})

    # mean session time
    temp12 = grpby_obj.previous_sessions_duration.median().\
                to_frame().reset_index().\
                rename(columns={'previous_sessions_duration': 'median_session_time'})

    
    for i in range(1, 13):
        temp = temp.merge(locals()[f'temp{i}'], 
                          on=grp_cols)
    return temp

In [13]:
def wrapper_agg(wst):
    sessions_ss = sessions[sessions.week_start == wst].copy()
    return weekly_aggregate(sessions_ss)

In [15]:
week_dates = sessions.week_start.dt.date.unique()

In [18]:
result = parallel(wrapper_agg, week_dates)
result = pd.concat(result)

In [21]:
result.head(2).T

,0,1
week_start,2018-10-01 00:00:00,2018-10-01 00:00:00
user_id_hash,000062e9be78f3da274fec338e78f89d12000e781967f2...,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...
no_city,5,2
city,campbell,douglasville
is_user_first_session,True,True
country,US,US
country_count,1,1
region,ca,ga
region_count,1,1
locale,en_US,en_US


In [22]:
result.to_pickle('../session_agg.pkl')

# Agg Events

In [75]:
gc.collect()

2325

In [76]:
events = pd.read_pickle('../events.pkl')

In [77]:
events.shape

(111946597, 9)

In [78]:
events.dtypes

session_id               category
event                    category
event_timestamp             int64
event_value               float64
user_id_hash             category
event_type               category
system_time        datetime64[ns]
system_date              category
week_start               category
dtype: object

In [80]:
events['system_time'] = events.event_timestamp.apply(lambda x: to_dt(x))

In [39]:
events.event.value_counts()

45                              75493931
1                                5363926
5                                4887922
6                                4068474
14                               3491117
4                                3435363
40                               2775997
7                                1970903
41                               1969223
3                                1828494
42                               1598212
.a5027911885258752                618037
.a5400102822346752                604699
44                                495590
.a5516611293544448                411053
0                                 407118
63                                347198
47                                310038
57                                271268
8                                 265034
55                                194341
.m5295687445250048                181875
.a5061295285075968                 92645
50                                 88641
9               

In [41]:
events['event_type'] = 'custom'

In [43]:
events.loc[events.event.str.startswith('.a'), ['event_type']] = 'ab'

In [44]:
events.loc[events.event.str.startswith('.c'), ['event_type']] = 'campaing'

In [45]:
events.loc[events.event.str.startswith('.m'), ['event_type']] = 'message'

In [46]:
events.event_type.value_counts()

custom      109627861
ab            1855428
message        463225
campaing           83
Name: event_type, dtype: int64

In [52]:
events.event_type = events.event_type.astype('category')

In [61]:
events.head()

,event,event_value,user_id_hash,event_type,system_time
0,45,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,custom,2018-11-14 17:09:57
1,45,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,custom,2018-11-14 17:11:24
2,.m5100869650219008,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,message,2018-11-02 02:06:50
3,4,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,custom,2018-12-02 01:11:31
4,6,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,custom,2018-12-02 01:11:33


In [9]:
from multiprocessing import Pool

In [37]:
num_partitions = 100
num_cores = 12
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [38]:
def wrapper_get_week_start(df):
    
    df['week_start'] = (df.system_time - \
                            df.system_time.dt.weekday.astype('timedelta64[D]')).dt.date
    return df

In [28]:
temp = parallelize_dataframe(events['system_time'].to_frame(), wrapper_get_week_start)

In [27]:
temp = parallelize_dataframe(events['system_time'].to_frame(), wrapper_get_week_start)
events['week_start'] = temp.week_start

In [29]:
events['week_start'] = events.week_start.astype('category')

In [35]:
events['system_date'] = events.system_date.astype('category')

In [30]:
events.dtypes

session_id               category
event                    category
event_timestamp             int64
event_value               float64
user_id_hash             category
event_type               category
system_time        datetime64[ns]
system_date                object
week_start               category
dtype: object

In [32]:
events.sort_values(['system_time', 'user_id_hash'], inplace=True)

In [66]:
events.reset_index(drop=True, inplace=True)

In [67]:
events.head()

,session_id,event,event_timestamp,event_value,user_id_hash,event_type,system_time,system_date,week_start
0,4339488202460077611,.a5027911885258752,1538377204526,0.0,c9bad12f865142f1b3938c9ab8a3003a7882136a2b4a12...,ab,2018-10-01 07:00:04,2018-10-01,2018-10-01
1,4339488202460077611,.a5061295285075968,1538377204526,0.0,c9bad12f865142f1b3938c9ab8a3003a7882136a2b4a12...,ab,2018-10-01 07:00:04,2018-10-01,2018-10-01
2,7012480646309837595,.a5027911885258752,1538377208476,0.0,285863e47df26bf47664c93d45c35ee3b9cfc8e253da23...,ab,2018-10-01 07:00:08,2018-10-01,2018-10-01
3,7012480646309837595,44,1538377240514,0.0,285863e47df26bf47664c93d45c35ee3b9cfc8e253da23...,custom,2018-10-01 07:00:40,2018-10-01,2018-10-01
4,7012480646309837595,5,1538377242744,0.0,285863e47df26bf47664c93d45c35ee3b9cfc8e253da23...,custom,2018-10-01 07:00:42,2018-10-01,2018-10-01


In [84]:
events.to_pickle('../events.pkl')

In [82]:
gc.collect()

971

In [85]:
events = pd.read_pickle('../events.pkl')

In [48]:
purchase = events.loc[events.event == '8', :].copy()

In [49]:
purchase.shape

(265034, 9)

In [50]:
purchase.to_pickle('../purchase.pkl')

In [51]:
events.shape

(111946597, 9)

In [86]:
events = events[events.event != '8']

In [87]:
events.shape

(111681563, 9)

In [71]:
events_sub = events[:1000].copy()

In [72]:
events_sub.event_value.describe()

count    1000.000000
mean        0.515000
std         2.874376
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        25.000000
Name: event_value, dtype: float64

In [73]:
events_sub.event_value.value_counts()

0.0     951
1.0      18
20.0     11
12.0      9
15.0      7
16.0      1
18.0      1
25.0      1
5.0       1
Name: event_value, dtype: int64

In [75]:
# events_sub.pivot_table(values='event_value', index=['week_start', 'user_id_hash'],
#                        columns='event_type', aggfunc='count')

In [11]:
gc.collect()

7

In [88]:
events.drop(['session_id', 'event_timestamp', 'system_time', 'system_date'], inplace=True, axis=1)

In [79]:
events.dtypes

event           category
event_value      float64
user_id_hash    category
event_type      category
week_start      category
dtype: object

In [89]:
events.event_type.value_counts()

custom      109362827
ab            1855428
message        463225
campaing           83
Name: event_type, dtype: int64

In [90]:
{'ab': 97, 'message': 98, 'campaing': 99}

{'ab': 97, 'message': 98, 'campaing': 99}

In [91]:
events.event = events.event.cat.add_categories([97, 98, 99])

In [92]:
events.loc[events.event_type == 'ab', 'event'] = 97
events.loc[events.event_type == 'message', 'event'] = 98
events.loc[events.event_type == 'campaing', 'event'] = 99

In [93]:
events.event = events.event.cat.remove_unused_categories()

In [108]:
events.event.value_counts()

45    75493931
1      5363926
5      4887922
6      4068474
14     3491117
4      3435363
40     2775997
7      1970903
41     1969223
97     1855428
3      1828494
42     1598212
44      495590
98      463225
0       407118
63      347198
47      310038
57      271268
55      194341
50       88641
9        85832
11       68328
64       67966
54       51687
43       23850
49       17955
48       11912
56        8267
52        6688
59        6651
60        6370
58        4167
10        2362
51        1446
61        1062
62         464
99          83
0/          35
1/          28
32           1
Name: event, dtype: int64

In [94]:
drop_events = ['56', '52', '59', '60', '58', '10', '51',
               '61', '62', '99', '0/', '1/', '32']

In [95]:
for e in drop_events:
    even = events[events.event != e]

In [112]:
events.dtypes

event           category
event_value      float64
user_id_hash    category
event_type      category
week_start      category
dtype: object

In [96]:
events.drop(['event_type'], inplace=True, axis=1)

In [115]:
events.dtypes

event           category
event_value      float64
user_id_hash    category
week_start      category
dtype: object

In [116]:
events.event_value.value_counts()

0.000000e+00    106109102
1.000000e+00      3629009
2.000000e+01       470187
1.200000e+01       357089
2.500000e+01       261102
1.500000e+01       199618
1.600000e+01       149336
1.700000e+01       139359
1.800000e+01       129954
1.900000e+01        92000
3.000000e+01        61014
5.000000e+00        55729
4.000000e+00         7594
1.400000e+01         6236
1.000000e+01         4281
3.000000e+00         3535
3.600000e+01         2215
7.000000e+00         2186
2.000000e+00          826
5.000000e+01          330
4.294967e+09          241
4.800000e+01          180
2.300000e+01          169
4.294967e+09           51
4.294967e+09           36
4.294967e+09           11
4.294967e+09            9
3.500000e+01            9
9.000000e+00            8
4.294967e+09            6
                  ...    
1.000000e+09            2
4.294967e+09            1
4.294967e+09            1
4.294973e+08            1
4.294967e+09            1
4.293967e+09            1
4.294967e+09            1
4.293967e+09

In [117]:
events_sub = events[:3000].copy()

In [119]:
events_sub = events_sub[events_sub.week_start == '2018-10-01']

In [97]:
def weekly_aggregate(df):

    grp_cols = ['week_start', 'event', 'user_id_hash']
    grpby_obj = df.groupby(grp_cols)
    
    temp = grpby_obj.event_value.count().to_frame().reset_index()\
                .rename(columns={'event_value': 'event_count'})
    temp1 = grpby_obj.event_value.sum().to_frame().reset_index()\
                .rename(columns={'event_value': 'event_sum'})
    temp2 = grpby_obj.event_value.mean().to_frame().reset_index()\
                .rename(columns={'event_value': 'event_mean'})

    for i in range(1, 3):
        temp = temp.merge(locals()[f'temp{i}'], 
                          on=grp_cols)
    return temp

In [126]:
result = weekly_aggregate(events_sub)

In [136]:
result.head()

,event,user_id_hash,event_count,event_sum,event_mean
0,0,ac6c2e659dbfbac273cd47c4a3aea9833e1b72e3afc8db...,1,0.0,0.0
1,0,be4f8aace99f98e60c4056822c54d141b1f53f58462231...,1,0.0,0.0
2,0,f8f57eeab62685b817353f70384952b6ee019e1738ebbd...,1,0.0,0.0
3,1,069fd1a9e916d4369e382e23e2aabda6409fe96ee60f93...,1,0.0,0.0
4,1,06f5949e56fcd2137cebf2ebe46044814816e22ff129bb...,1,0.0,0.0


In [128]:
result.event_mean.describe()

count    875.000000
mean       1.430933
std        4.828495
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       30.000000
Name: event_mean, dtype: float64

In [129]:
result.event_sum.describe()

count    875.000000
mean       2.014857
std        7.595374
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       65.000000
Name: event_sum, dtype: float64

In [98]:
week_dates = events.week_start.dt.date.unique()

In [99]:
gc.collect()

7

In [100]:
def wrapper_agg(wst):
    events_ss = events[events.week_start == wst].copy()
    return weekly_aggregate(events_ss)

result = parallel(wrapper_agg, week_dates)
result = pd.concat(result)

In [101]:
result.shape

(8029501, 6)

In [102]:
result.head()

,week_start,event,user_id_hash,event_count,event_sum,event_mean
0,2018-10-01,0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,1,0.0,0.0
1,2018-10-01,0,000c653971c4cdb4a930ebb9de12c8587f8c46cdf1bdf5...,1,0.0,0.0
2,2018-10-01,0,001341dc1ffc939af25106b66a54ea318c15a03d7d699a...,1,0.0,0.0
3,2018-10-01,0,001649233906501efe5899fc693500a233220cfc02be0d...,1,0.0,0.0
4,2018-10-01,0,001b256fc745aff7b582d02dd1e7d9209a7b6c486b4cf2...,1,0.0,0.0


In [103]:
result.to_pickle('../events_agg.pkl')

# Agg Atrributes

## combining

In [11]:
attrib = pd.read_pickle('../../attributes2.pkl')

In [12]:
attrib.dtypes

session_id         category
attribute             uint8
attribute_value      object
user_id_hash       category
dtype: object

In [13]:
attrib.shape

(185590092, 4)

In [15]:
attrib.head()

,session_id,attribute,attribute_value,user_id_hash
0,2201961907282901522,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2201961907282901522,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,2201961907282901522,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2201961907282901522,4,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2201961907282901522,5,romance_the_royal_romance_02:11,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [14]:
events = pd.read_pickle('../data/sessions.pkl')

In [31]:
events.head()

,session_id,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,user_id_hash,user_time,system_time,system_date,anchor_date
0,4339488202460077611,Australia/Sydney,36000000,0,2018-10-01 07:00:04,True,AU,qld,gold coast,-28.016666,153.399994,en-AU_AU,iPhone OS,1,c9bad12f865142f1b3938c9ab8a3003a7882136a2b4a12...,17,2018-10-01 07:00:04,2018-10-01,2018-10-01
1,7012480646309837595,Pacific/Auckland,46800000,0,2018-10-01 07:00:08,True,NZ,auk,auckland,-36.848461,174.763336,en-NZ_NZ,iOS,1,285863e47df26bf47664c93d45c35ee3b9cfc8e253da23...,20,2018-10-01 07:00:08,2018-10-01,2018-10-01
2,2920311224184894615,Asia/Kolkata,19800000,1000637,2018-10-01 06:44:08,False,IN,tg,hyderabad,17.385044,78.486671,en_US,Android OS,2,475f8f20576637055c5331513dda4db1d889f5e951c8e5...,12,2018-10-01 07:01:02,2018-10-01,2018-10-01
3,8483277250870486701,America/Chicago,-18000000,0,2018-10-01 07:01:30,True,US,tx,houston,29.760427,-95.369804,en_US,Android OS,1,4bc7b52d56b24d258ba1746994492f354fd4b17fcaae81...,2,2018-10-01 07:01:30,2018-10-01,2018-10-01
4,8114762104527844914,America/Los_Angeles,-25200000,2415582,2018-10-01 06:18:23,False,US,ca,sacramento,38.581573,-121.494400,en_US,Android OS,2,82349b0d1a80b42e59519068ded46178778084b9602b9c...,0,2018-10-01 07:01:57,2018-10-01,2018-10-01


In [15]:
events = events.loc[:, ['session_id', 'system_time']]

In [33]:
events.shape

(5165760, 2)

In [16]:
events.drop_duplicates(inplace=True)

In [35]:
events.shape

(5165730, 2)

In [17]:
events.reset_index(inplace=True, drop=True)

In [18]:
events.session_id = events.session_id.astype('category')

In [38]:
events.head()

,session_id,system_time
0,4339488202460077611,2018-10-01 07:00:04
1,7012480646309837595,2018-10-01 07:00:08
2,2920311224184894615,2018-10-01 07:01:02
3,8483277250870486701,2018-10-01 07:01:30
4,8114762104527844914,2018-10-01 07:01:57


In [19]:
events_d = events.groupby(['session_id'])['system_time'].last().to_frame()

In [20]:
events_d.reset_index(inplace=True)

In [41]:
events_d.head()

,session_id,system_time
0,1000000874609429755,NaT
1,100000246528643894,2018-11-07 14:45:09
2,1000003546493394404,2018-11-06 13:05:19
3,1000004963329986136,2018-11-08 07:37:13
4,1000005015636510449,2018-11-10 20:25:09


In [21]:
events = events_d

In [22]:
import gc
gc.collect()

35

In [44]:
events.dtypes

session_id           category
system_time    datetime64[ns]
dtype: object

In [45]:
attrib.dtypes

session_id         category
attribute             uint8
attribute_value      object
user_id_hash       category
dtype: object

In [23]:
attrib.session_id = attrib.session_id.astype(np.int64)
events.session_id = events.session_id.astype(np.int64)

In [24]:
attrib = attrib.merge(events, how='left' ,on=['session_id'])

In [25]:
attrib.shape

(185590092, 5)

In [49]:
attrib.head()

,session_id,attribute,attribute_value,user_id_hash,system_time
0,2201961907282901522,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
1,2201961907282901522,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
2,2201961907282901522,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
3,2201961907282901522,4,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
4,2201961907282901522,5,romance_the_royal_romance_02:11,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37


In [26]:
del events

In [27]:
attrib.drop(['session_id'], inplace=True, axis=1)

In [28]:
gc.collect()

28

In [39]:
attrib.to_pickle('../attributes3.pkl')

In [50]:
attrib = pd.read_pickle('../attributes3.pkl')

In [54]:
attrib.head()

,attribute,attribute_value,user_id_hash,system_time
0,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
1,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
2,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
3,4,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37
4,5,romance_the_royal_romance_02:11,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02 01:09:37


In [29]:
attrib['system_date'] = attrib.system_time.dt.date

In [30]:
attrib.attribute = attrib.attribute.astype('category')

In [57]:
attrib.dtypes

attribute                category
attribute_value            object
user_id_hash             category
system_time        datetime64[ns]
system_date                object
dtype: object

In [31]:
attrib.drop(['system_time'], inplace=True, axis=1)

In [59]:
attrib.head()

,attribute,attribute_value,user_id_hash,system_date
0,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
1,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
2,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
3,4,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
4,5,romance_the_royal_romance_02:11,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02


## Attributes EDA

In [49]:
attrib.attribute.value_counts()

16    6776351
15    6776351
1     6776351
3     6776351
4     6776351
5     6776351
6     6776351
7     6776351
8     6776351
9     6776351
10    6776351
11    6776351
13    6776351
14    6776351
0     6776351
17    6776351
18    6776351
19    6776351
20    6776351
21    6776351
52    6776351
75    6776297
76    6776297
77    6776297
78    6685459
66    6614104
48    6183014
67    3395545
96      79643
49         65
Name: attribute, dtype: int64

In [25]:
for a in attrib.attribute.unique():
    print(a)
    try:
        print(attrib.loc[attrib.attribute == a, :].attribute_value.unique())
    except:
        print('dict')

0
[1 0 2 3 4 6 15 9 10 7 11 8 5 14 12 18 16 13 17 19 27 20 21 91 33 56 23 22
 26 25 28 30 31 29 24 40 37 39 36 35 38 32 34 47 44 51 41 52 43 50 46 70
 187 170 76 42 45 48 60 77 85 55 75 58 69 87 94 66 49 81 73 62 93 57 59 80
 86 61 97 53 74 177 100 54 63 84 71 112 166 178 64 117 88 68 195 78 105
 181 167 179 131 114 155 83 107 137 109 82 102 67 165 92 72 65 79 108 126
 90 101 161 89 154 182 160]
1
[1 0 9 ... 1743 1727 1692]
3
[0 6 10 5 4 1 2 3 22 12 24 20 28 23 25 16 19 13 11 18 21 9 27 8 26 29 7 14
 17 15 30 34 32 104 107 59 142 109 49 72 86 68 110 127 119 135 120 96 67
 122 146 47 121 35 55 89 103 31 101 105 74 39 148 139 133 157 108 95 114
 116 126 134 143 117 123 112 141 155 152 33 94 79 124 128 125 88 54 75 50
 87 82 83 69 147 84 91 118 76 97 115 46 106 140 71 41 77 43 78 93 149 144
 136 145 270 268 267 232 281 214 295 294 209 241 211 230 239 238 210 225
 298 219 222 284 299 215 256 296 245 261 260 272 300 259 249 221 99 61 131
 132 85 52 60 98 102 100 80 129 62 63 53 58 81 51 130

dict
8
[2 1 3 5 6 4 0 18 16 14 15 7 10 17 11 8 9 29 13 12 21 20 19 28 22 39 30 25
 32 26 27 33 31 23 24 34 48 44 47 40 36 37 35 41 38 46 43 42 51 50 49 45
 56 54 62 61 52 60 53 58 55]
9
[29 8 21 ... 1081 1429 871]
10
dict
11
[0 1 6 178 20 132 17 73 41 2 11 3 47 196 16 40 136 114 56 4 43 10 49 604
 65 32 134 22 155 151 303 37 7 55 191 103 23 51 77 12 39 9 14 327 18 36 25
 59 29 254 316 27 5 305 380 33 135 123 19 90 46 62 105 35 52 167 112 30 97
 48 26 8 15 53 13 156 91 150 442 117 204 21 169 187 111 31 171 113 168 125
 44 568 325 107 54 139 69 24 206 418 232 100 84 347 267 154 80 50 429 28
 104 148 106 67 78 71 118 280 338 430 236 92 42 256 361 99 409 314 482 149
 45 364 82 75 489 63 159 98 58 174 137 93 72 121 127 290 89 162 101 312
 366 271 94 355 34 102 164 74 170 95 86 142 126 138 261 257 288 96 68 499
 402 144 79 414 502 200 124 210 57 252 382 235 297 268 182 721 362 340 215
 408 208 247 606 464 110 189 692 129 318 379 300 64 209 61 83 374 240 38
 691 241 251 354 192 197 222 618 60

[0 14 20 18 65 3 50 22 38 46 8 45 13 16 30 10 25 9 15 11 28 53 34 1 33 47
 5 23 35 36 4 19 21 2 17 37 26 12 7 48 6 41 40 29 27 31 24 32 49 39 43 55
 42 51 63 9999 999999 58 999]
19
[16 13 10 9 4 7 18 6 2 20 8 0 12 26 17 22 32 34 5 23 19 21 29 28 14 27 25
 31 24 3 15 1 35 11 33 46 37 42 45 44 40 39 36 41 30 38 147 66 76 55 54 64
 60 62 56 47 74 59 52 53 69 61 58 57 68 49 71 50 75 51 72 43 48 83 93 107
 182 180 118 167 101 111 65 142 78 171 177 159 115 145 148 113 99 95 97 90
 80 82 153 164 67 105 174 128 162 73 125 134 131 123 92 121 151 86 88 89
 135 122 94 91 112 87 169 191 117 84 119 179 120 96 79 197 187 221 109 149
 181 165 129 100 77 199 108 106 81 189 193 205 175 126 215 185 124 209 207
 213 98 103 104 225 70 223 155 116 141 161 110 132 137 130 127 173 133 114
 102 409 448 357 442 388 235 384 361 431 351 333 436 400 204 244 360 429
 234 477 392 413 345 415 453 454 471 355 353 460 469 401 405 450 327 425
 492 354 325 317 439 398 385 463 428 346 331 336 426 438 296 427 481 424
 432

[1 0]
96
[1 0]
67
[1 0]
49
[1.0]


In [27]:
for a in attrib.attribute.unique():
    try:
        print(a, len(attrib.loc[attrib.attribute == a, :].attribute_value.unique()))
    except:
        pass

0 124
1 1685
3 270
4 933
6 698
8 61
9 1269
11 579
13 29736
14 14466
15 389
16 995
17 230
18 59
19 868
20 5
21 5
75 657
76 13
77 657
78 2
96 2
67 2
49 1


|attribute| type | counts|
|---|--------------|-----|
| 0 | list of ints | 124 |
| 1 | list of ints | 1685 |
| 3 | list of ints | 270 |
| 6 | list of ints | 698 |
| 8 | list of ints | 61 |
| 9 | list of ints | 1269 |
| 11 | list of ints | 579 |
| 15 | list of ints | 389 |
| 16 | list of ints | 995 |
| 17 | list of ints | 230 |
| 18 | list of ints | 59 |
| 19 | list of ints | 868 |
| 75 | list of ints | 657 |
| 77 | list of ints | 675 |
| 4 | list of floats | 933 |
| 13 | list of floats | 29736 |
| 14 | list of flaots | 14466 |
| 5 | dict | |
| 7 | dict | |
| 10 | dict | |
| 48 | dict | |
| 52 | dict | |
| 66 | dict | |
| 20 | list of ints cats | 5 |
| 21 | list of ints cats | 5 |
| 76 | list of ints cats | 13 |
| 78 | bool | 2 |
| 96 | bool | 2 |
| 67 | bool | 2 |
| 49 | bool | 1 |


In [32]:
def convert_5(x):
    try:
        t = x.split(':')
        return int(t[1])
    except:
        return None

In [33]:
attrib.loc[attrib.attribute == 5, ['attribute_value']] =\
    attrib[attrib.attribute == 5].attribute_value.apply(lambda x: convert_5(x))

In [62]:
attrib.loc[attrib.attribute == 5, ['attribute_value']].tail()

,attribute_value
185589961,1
185589992,1
185589993,1
185590046,1
185590047,1


In [34]:
def convert_7(x):
    try:
        t = x.split(':')
        return int(t[1])
    except:
        return None

In [35]:
attrib.loc[attrib.attribute == 7, ['attribute_value']] = \
    attrib[attrib.attribute == 7].attribute_value.apply(lambda x: convert_7(x))

In [65]:
attrib.loc[attrib.attribute == 7, ['attribute_value']].tail()

,attribute_value
185589963,4
185589996,NaN
185589997,NaN
185590050,NaN
185590051,NaN


In [36]:
def convert_10(x):
    try:
        t = x.split(':')
        return int(t[1])
    except:
        return None

In [37]:
attrib.loc[attrib.attribute == 10, ['attribute_value']] = \
    attrib[attrib.attribute == 10].attribute_value.apply(lambda x: convert_10(x))

In [68]:
attrib.loc[attrib.attribute == 10, ['attribute_value']].tail()

,attribute_value
185589966,1
185590002,NaN
185590003,NaN
185590056,NaN
185590057,NaN


In [38]:
attrib.loc[attrib.attribute == 48, ['attribute_value']] = 1

In [39]:
attrib.loc[attrib.attribute == 52, ['attribute_value']] = 1

In [41]:
def convert_66(x):
    try:
        t = x[2:-2].replace('\\', '').replace('"', '').split(',')
        return max([int(d.split(':')[1]) for d in t])
    except:
        return None

In [42]:
attrib.loc[attrib.attribute == 66, ['attribute_value']] = \
    attrib[attrib.attribute == 66].attribute_value.apply(lambda x: convert_66(x))

In [74]:
attrib.dtypes

attribute          category
attribute_value      object
user_id_hash       category
system_date          object
dtype: object

In [75]:
attrib.shape

(185590092, 4)

In [43]:
attrib = attrib[(~attrib.attribute.isin([49, 48, 52]))]

In [45]:
gc.collect()

342

In [46]:
attrib.shape

(172630662, 4)

In [47]:
attrib.dtypes

attribute          category
attribute_value      object
user_id_hash       category
system_date          object
dtype: object

In [48]:
attrib.head()

,attribute,attribute_value,user_id_hash,system_date
0,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
1,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
2,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
3,4,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
4,5,11,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02


count of all

mean & max for 4, 13, 14

median of 20, 21

mean of 78, 96, 67

## Atrrib Agg

In [78]:
del_atr = list(set(list(attrib.attribute.unique())) -\
                   set([4, 13, 14, 20, 21, 78, 96, 67]))

In [79]:
for d in del_atr:
    attrib = attrib[attrib.attribute != d]

In [80]:
gc.collect()

28

In [81]:
attrib.shape

(44042402, 4)

In [116]:
week_dates = attrib.week_start.dt.date.unique()

In [62]:
attrib.head()

,attribute,attribute_value,user_id_hash,system_date
0,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
1,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
2,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
3,4,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02
4,5,11,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2018-12-02


In [76]:
attrib.reset_index(inplace=True, drop=True)

In [64]:
attrib.sort_values(['system_date', 'user_id_hash'], inplace=True)

In [66]:
gc.collect()

8

In [49]:
attrib.attribute_value = attrib.attribute_value.astype(np.number)

In [50]:
attrib.system_date = attrib.system_date.astype('category')

In [51]:
attrib.dtypes

attribute          category
attribute_value     float64
user_id_hash       category
system_date        category
dtype: object

In [52]:
gc.collect()

0

In [73]:
attrib.to_pickle('../temp')

In [74]:
attrib = pd.read_pickle('../temp')

In [77]:
attrib.head()

,attribute,attribute_value,user_id_hash,system_date
0,0,0.0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,2018-10-01
1,1,0.0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,2018-10-01
2,3,0.0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,2018-10-01
3,4,0.0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,2018-10-01
4,5,1.0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,2018-10-01


In [78]:
attrib.shape

(172630662, 4)

In [53]:
attrib.to_pickle('../data/atr.pkl')

In [84]:
time_df = attrib.system_date.dt.date.unique()

In [85]:
time_df = pd.DataFrame({'system_time': time_df})

In [86]:
time_df.system_time = time_df.system_time.astype('datetime64[ns]')

In [88]:
time_df = time_df.loc[:74, :]

In [89]:
time_df

,system_time
0,2018-10-01
1,2018-10-02
2,2018-10-03
3,2018-10-04
4,2018-10-05
5,2018-10-06
6,2018-10-07
7,2018-10-08
8,2018-10-09
9,2018-10-10


In [90]:
temp = dict(zip(time_df.system_time.dt.date.values,
                (time_df.system_time - time_df.system_time.dt.weekday\
                 .apply(lambda x: dt.timedelta(days=(x+2) % 7)))\
                .dt.date.values))

In [91]:
attrib.system_date = attrib.system_date.dt.date

In [93]:
attrib.isna().sum()

attribute                 0
attribute_value     1789233
user_id_hash              0
system_date        35126531
dtype: int64

In [95]:
attrib.shape

(172630662, 4)

In [96]:
attrib = attrib[~attrib.system_date.isna()]

In [97]:
attrib.shape

(137504131, 4)

In [98]:
attrib['anchor_date'] = attrib.system_date.apply(lambda x: temp[x])

In [99]:
attrib.attribute_value.isna().sum()

1319450

In [100]:
attrib = attrib[~attrib.attribute_value.isna()]

In [106]:
week_dates = attrib.anchor_date.unique()

In [101]:
def weekly_aggregate(df):

    grp_cols = ['anchor_date', 'attribute', 'user_id_hash']
    grpby_obj = df.groupby(grp_cols)
    
    temp = grpby_obj.attribute_value.count().to_frame().reset_index()\
                .rename(columns={'attribute_value': 'atr_count'})
    temp1 = grpby_obj.attribute_value.mean().to_frame().reset_index()\
                .rename(columns={'attribute_value': 'atr_mean'})
    temp2 = grpby_obj.attribute_value.last().to_frame().reset_index()\
                .rename(columns={'attribute_value': 'atr_last'})
    temp3 = grpby_obj.attribute_value.max().to_frame().reset_index()\
                .rename(columns={'attribute_value': 'atr_max'})

    for i in range(1, 4):
        temp = temp.merge(locals()[f'temp{i}'], 
                          on=grp_cols)
    return temp

In [105]:
def wrapper_agg(wst):
    events_ss = attrib[attrib.anchor_date == wst].copy()
    return weekly_aggregate(events_ss)

In [103]:
attrib.shape

(136184681, 5)

In [104]:
attrib.attribute_value.apply(np.isreal).sum()

136184681

In [124]:
attrib.attribute_value = attrib.attribute_value.astype(np.number)

In [107]:
result = parallel(wrapper_agg, week_dates)
result = pd.concat(result)

In [108]:
result.head()

,anchor_date,attribute,user_id_hash,atr_count,atr_mean,atr_last,atr_max
0,2018-09-29,0,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,0.0,0.0,0.0
1,2018-09-29,0,0003f8bda56230a49445880e559b718e94ba37344494a7...,1,0.0,0.0,0.0
2,2018-09-29,0,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1,0.0,0.0,0.0
3,2018-09-29,0,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,3,0.0,0.0,0.0
4,2018-09-29,0,0007da1b2aa0df73cf98077068a7da9039f57d9d51a1ae...,1,0.0,0.0,0.0


In [109]:
result.to_pickle('../attributes_agg2.pkl')

In [110]:
attrib.to_pickle('../temp2')

# Messages

In [41]:
msgs = pd.read_csv('../messages.csv')

In [42]:
msgs.head()

,app_id,message_id,action_type,delivery_type,delivery_time_mode,goal_kind
0,6196435404455936,5420304779837440,__Push Notification,4,3,NaN
1,6196435404455936,5059935179767808,__Push Notification,0,3,NaN
2,6196435404455936,6563799154425856,__Webhook,4,3,NaN
3,6196435404455936,4680497250304000,__Push Notification,0,3,NaN
4,4724682771660800,6286714069450752,Center Popup,0,3,3.0


In [44]:
msgs.shape

(2896, 6)

# Combining Everything

In [11]:
events = pd.read_pickle('../events_agg.pkl')
session = pd.read_pickle('../session_agg.pkl')
atr = pd.read_pickle('../attributes_agg.pkl')
user_date = pd.read_pickle('../user_date.pkl')

In [12]:
df = session

In [13]:
user_date = user_date.groupby('user_id_hash').user_created_timestamp.max()

In [14]:
user_date = user_date.to_frame()

In [15]:
user_date.reset_index(inplace=True)

In [16]:
df = df.merge(user_date, on=['user_id_hash'], how='left')

In [17]:
events.head()

,week_start,event,user_id_hash,event_count,event_sum,event_mean
0,2018-10-01,0,0008ae1788be9473a11dd8efdc61010ad2a54ffe5e17ad...,1,0.0,0.0
1,2018-10-01,0,000c653971c4cdb4a930ebb9de12c8587f8c46cdf1bdf5...,1,0.0,0.0
2,2018-10-01,0,001341dc1ffc939af25106b66a54ea318c15a03d7d699a...,1,0.0,0.0
3,2018-10-01,0,001649233906501efe5899fc693500a233220cfc02be0d...,1,0.0,0.0
4,2018-10-01,0,001b256fc745aff7b582d02dd1e7d9209a7b6c486b4cf2...,1,0.0,0.0


In [18]:
events.set_index(['week_start', 'user_id_hash', 'event'], inplace=True)

In [19]:
events_p = events.unstack()

In [20]:
events_p.fillna(-99, inplace=True)

In [21]:
events_p.columns = [i + str(j) for  i, j  in zip (events_p.columns.get_level_values(0), events_p.columns.get_level_values(1))]

In [22]:
events_p.head()

event_count0  \
week_start user_id_hash                                                       
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...         -99.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...         -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...         -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...         -99.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...         -99.0   

                                                               event_count0/  \
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...          -99.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...          -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...          -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...          -99.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...          -99.0   

                                                               event_count1  \
week_start user_id_hash                                                       
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...           1.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...         -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...         -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...           1.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...           2.0   

                                                               event_count1/  \
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...          -99.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...          -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...          -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...          -99.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...          -99.0   

                                                               event_count10  \
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...          -99.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...          -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...          -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...          -99.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...          -99.0   

                                                               event_count11  \
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...          -99.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...          -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...          -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...          -99.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...          -99.0   

                                                               event_count14  \
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...            3.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...            3.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...            3.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...            4.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...            4.0   

                                                               event_count3  \
week_start user_id_hash                                                

In [23]:
events_p.reset_index(inplace=True)

In [24]:
events_p.head()

,week_start,user_id_hash,event_count0,event_count0/,event_count1,event_count1/,event_count10,event_count11,event_count14,event_count3,...,event_mean60,event_mean61,event_mean62,event_mean63,event_mean64,event_mean7,event_mean9,event_mean97,event_mean98,event_mean99
0,2018-10-01,000062e9be78f3da274fec338e78f89d12000e781967f2...,-99.0,-99.0,1.0,-99.0,-99.0,-99.0,3.0,2.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,20.0,-99.0,0.0,-99.0,-99.0
1,2018-10-01,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,3.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,12.0,-99.0,0.0,-99.0,-99.0
2,2018-10-01,0003f8bda56230a49445880e559b718e94ba37344494a7...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,3.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,-99.0,-99.0
3,2018-10-01,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,-99.0,-99.0,1.0,-99.0,-99.0,-99.0,4.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,12.0,-99.0,0.0,-99.0,-99.0
4,2018-10-01,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,-99.0,-99.0,2.0,-99.0,-99.0,-99.0,4.0,-99.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,21.0,-99.0,0.0,-99.0,-99.0


In [25]:
events_p.shape

(1152353, 122)

In [26]:
df.week_start = df.week_start.astype('category')

In [27]:
df = df.merge(events_p, on=['user_id_hash', 'week_start'], how='left')

In [28]:
df.shape

(995976, 136)

In [30]:
atr.attribute = atr.attribute.astype(np.int)

In [31]:
atr.head()

,atr_count,atr_last,atr_max,atr_mean,attribute,user_id_hash,week_start
0,1,0.05,0.05,0.05,4,000059859ec188af6035870faf885c3038cedda05b3a54...,2018-11-26 00:00:00
1,1,0.00,0.00,0.00,4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,2018-11-26 00:00:00
2,8,0.00,0.00,0.00,4,0000bc4e8bfc829d327fa69bdc426dd8a789c52550ed87...,2018-11-26 00:00:00
3,1,0.00,0.00,0.00,4,000161848e5ccdcd4d3b0c7be9ec844730c39e6bcbc2a7...,2018-11-26 00:00:00
4,1,0.00,0.00,0.00,4,00020102f66f89759868c843ae9fe634870f132f9e3d79...,2018-11-26 00:00:00


In [32]:
atr.set_index(['week_start', 'user_id_hash', 'attribute'], inplace=True)

In [33]:
atr_p = atr.unstack()

In [34]:
atr_p.fillna(-99, inplace=True)

In [35]:
atr_p.head()

atr_count       \
attribute                                                            4    13   
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...       4.0  4.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...       2.0  2.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...       1.0  1.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...       1.0  1.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...       4.0  4.0   

                                                                              \
attribute                                                       14   20   21   
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...  4.0  4.0  4.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...  2.0  2.0  2.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...  1.0  1.0  1.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...  1.0  1.0  1.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...  4.0  4.0  4.0   

                                                                          \
attribute                                                        67   78   
week_start user_id_hash                                                    
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23... -99.0  4.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3... -99.0  2.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f... -99.0  1.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9... -99.0  1.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87... -99.0  4.0   

                                                                    atr_last  \
attribute                                                        96       4    
week_start user_id_hash                                                        
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23... -99.0     0.00   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3... -99.0     0.00   
           0003f8bda56230a49445880e559b718e94ba37344494a7f... -99.0     0.00   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9... -99.0     0.02   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87... -99.0     0.00   

                                                                     ...   \
attribute                                                        13  ...    
week_start user_id_hash                                              ...    
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...  0.00  ...    
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...  0.00  ...    
           0003f8bda56230a49445880e559b718e94ba37344494a7f...  0.00  ...    
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...  0.02  ...    
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...  0.00  ...    

                                                              atr_max        \
attribute                                                          78    96   
week_start user_id_hash                                                       
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23...     1.0 -99.0   
           00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd3...     1.0 -99.0   
           0003f8bda56230a49445880e559b718e94ba37344494a7f...     1.0 -99.0   
           000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b9...     1.0 -99.0   
           0006028dc3aefd41499be9c5428fb11691f3ce570e7cc87...     1.0 -99.0   

                                                              atr_mean  \
attribute                                                           4    
week_start user_id_hash                                                  
2018-10-01 000062e9be78f3da274fec338e78f89d12000e781967f23

In [36]:
atr_p.columns = [i + str(j) for  i, j  in zip (atr_p.columns.get_level_values(0),
                                               atr_p.columns.get_level_values(1))]

In [37]:
atr_p.reset_index(inplace=True)

In [38]:
atr_p.head()

,week_start,user_id_hash,atr_count4,atr_count13,atr_count14,atr_count20,atr_count21,atr_count67,atr_count78,atr_count96,...,atr_max78,atr_max96,atr_mean4,atr_mean13,atr_mean14,atr_mean20,atr_mean21,atr_mean67,atr_mean78,atr_mean96
0,2018-10-01,000062e9be78f3da274fec338e78f89d12000e781967f2...,4.0,4.0,4.0,4.0,4.0,-99.0,4.0,-99.0,...,1.0,-99.0,0.0075,0.0075,0.0,0.0,1.0,-99.0,1.0,-99.0
1,2018-10-01,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2.0,2.0,2.0,2.0,2.0,-99.0,2.0,-99.0,...,1.0,-99.0,0.0000,0.0000,0.0,0.0,2.0,-99.0,1.0,-99.0
2,2018-10-01,0003f8bda56230a49445880e559b718e94ba37344494a7...,1.0,1.0,1.0,1.0,1.0,-99.0,1.0,-99.0,...,1.0,-99.0,0.0000,0.0000,0.0,0.0,2.0,-99.0,1.0,-99.0
3,2018-10-01,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1.0,1.0,1.0,1.0,1.0,-99.0,1.0,-99.0,...,1.0,-99.0,0.0200,0.0200,0.0,2.0,0.0,-99.0,1.0,-99.0
4,2018-10-01,0006028dc3aefd41499be9c5428fb11691f3ce570e7cc8...,4.0,4.0,4.0,4.0,4.0,-99.0,4.0,-99.0,...,1.0,-99.0,0.0000,0.0000,0.0,0.5,0.0,-99.0,1.0,-99.0


In [44]:
atr_p.week_start = atr_p.week_start.astype('category')

In [45]:
df = df.merge(atr_p, on=['user_id_hash', 'week_start'], how='left')

In [46]:
df.shape

(995976, 168)

In [47]:
df.to_pickle('../merged.pkl')

In [229]:
df = pd.read_pickle('../merged.pkl')

# Preparing for Model

In [230]:
purchase = pd.read_pickle('../purchase.pkl')

In [231]:
purchase.reset_index(inplace=True, drop=True)

In [232]:
t1 = purchase.groupby(['week_start', 'user_id_hash']).event_value.count()\
            .reset_index().rename(columns={'event_value': 'no_of_buys'})
t2 = purchase.groupby(['week_start', 'user_id_hash']).event_value.sum()\
            .reset_index().rename(columns={'event_value': 'sum_of_buys'})

In [233]:
purchase = t1.merge(t2, on=['week_start', 'user_id_hash'])

In [234]:
purchase.head()

,week_start,user_id_hash,no_of_buys,sum_of_buys
0,2018-10-01,001c2d06972b17ee6a075ff9fbdae33eccb0bbf28357ea...,1,1.393000
1,2018-10-01,0026497d2043c8530220eeb8e66051982cc6df3139a9b0...,3,10.478999
2,2018-10-01,006b49314baf054c636e7448d4f9b0df046aae9e9c17d0...,10,20.229999
3,2018-10-01,007327aa3a27718c76e6cd73c3971304280c8856f62940...,1,1.393000
4,2018-10-01,0088d9be17b2ec4b3621150117a0ede815547d4241eae1...,2,6.986000


In [235]:
purchase1 = purchase.copy(deep=True)
purchase1.week_start = purchase1.week_start.apply(lambda x: x- dt.timedelta(days=7))

In [236]:
purchase2 = purchase.copy(deep=True)
purchase2.week_start = purchase2.week_start.apply(lambda x: x - dt.timedelta(days=14))

In [237]:
df = df.merge(purchase, on=['user_id_hash', 'week_start'], how='left')

In [238]:
df.shape

(995976, 170)

In [239]:
df.no_of_buys.fillna(0, inplace=True)
df.sum_of_buys.fillna(0, inplace=True)

In [240]:
df = df.merge(purchase1, on=['user_id_hash', 'week_start'], how='left')

In [241]:
df.drop(['sum_of_buys_y'], inplace=True, axis=1)
df.rename(columns={'no_of_buys_x': 'no_of_buys',
                    'sum_of_buys_x': 'sum_of_buys',
                    'no_of_buys_y': 'y1'}, inplace=True)

In [242]:
df.week_start = df.week_start.astype('category')

In [243]:
df = df.merge(purchase2, on=['user_id_hash', 'week_start'], how='left')

In [244]:
df.drop(['sum_of_buys_y'], inplace=True, axis=1)
df.rename(columns={'no_of_buys_x': 'no_of_buys',
                    'sum_of_buys_x': 'sum_of_buys',
                    'no_of_buys_y': 'y2'}, inplace=True)

In [245]:
df['y1'].fillna(0, inplace=True)
df['y2'].fillna(0, inplace=True)
df['y2'] = df['y1'] + df['y2']
df['y1'] = df['y1'] > 0
df['y2'] = df['y2'] > 0

# Test Train Split

In [246]:
from sklearn.model_selection import KFold

def reg_target_encoding(train, col = "device_type", splits=5):
    """ Computes regularize mean encoding.
    Inputs:
       train: training dataframe
       
    """
    ksplit = KFold(n_splits=splits)
    for tidx, vidx in ksplit.split(train):
        mean_val = train.iloc[tidx, :].groupby(col)['y1'].mean()
        global_mean_val = train.iloc[tidx, :]['y1'].mean()
        train.loc[vidx, col + "_mean_enc"] = train.iloc[vidx, :][col].map(mean_val)
        train.loc[vidx, col + "_mean_enc"].fillna(global_mean_val, inplace=True)
    return None

In [247]:
def mean_encoding_test(test, train, col = "device_type"):
    """ Computes target enconding for test data.
    
    This is similar to how we do validation
    """
    mean_device_type = train.groupby(col)['y1'].mean()
    global_mean = train['y1'].mean()
    test[col + "_mean_enc"] = test[col].map(mean_device_type)
    test[col + "_mean_enc"].fillna(global_mean, inplace=True)
    return None

In [248]:
df.city = df.city.astype('category')
df.country = df.country.astype('category')
df.region = df.region.astype('category')
df.locale = df.locale.astype('category')
df.os_name = df.os_name.astype('category')

In [249]:
for c in [i for i in df.columns if i.startswith('atr_') or i.startswith('event_')]:
    df[c].fillna(-99, inplace=True)

In [250]:
df['user_age'] = (df.week_start.dt.date - df.user_created_timestamp)\
                    .astype('timedelta64[D]')

In [251]:
train = df[df.week_start < '2018, 12, 11']
test = df[df.week_start >= '2018, 12, 11']

In [252]:
reg_target_encoding(train, col='country')
mean_encoding_test(test, train, col='country')

In [253]:
reg_target_encoding(train, col='week_start')
mean_encoding_test(test, train, col='week_start')

In [254]:
reg_target_encoding(train, col='region')
mean_encoding_test(test, train, col='region')

In [255]:
train.city = train.city.cat.codes
train.country = train.country.cat.codes
train.region = train.region.cat.codes
train.os_name = train.os_name.cat.codes
train.locale = train.locale.cat.codes

In [256]:
test.city = test.city.cat.codes
test.country = test.country.cat.codes
test.region = test.region.cat.codes
test.os_name = test.os_name.cat.codes
test.locale = test.locale.cat.codes

In [257]:
train.is_user_first_session = train.is_user_first_session.astype(np.int)
train['y1'] = train['y1'].astype(np.int)
train['y2'] = train['y2'].astype(np.int)

In [258]:
test.is_user_first_session = test.is_user_first_session.astype(np.int)
test['y1'] = test['y1'].astype(np.int)
test['y2'] = test['y2'].astype(np.int)

In [259]:
train.shape

(995976, 176)

In [260]:
test.shape

(0, 176)

In [261]:
feature_cols = ['no_city', 'city', 'is_user_first_session',
                'country', 'country_count', 'region', 'region_count',
                'locale', 'os_name', 'user_days', 'total_session_time',
                'mean_session_time', 'median_session_time',
                'user_age', 'country_mean_enc', 'no_of_buys',
                'sum_of_buys', 'week_start_mean_enc', 'region_mean_enc']

feature_cols += [i for i in df.columns if i.startswith('atr_') or i.startswith('event_')]

In [262]:
x_train = train[feature_cols].values
y1_train = train[['y1']]
y2_train = train[['y2']]

x_test = test[feature_cols].values
y1_test = test[['y1']]
y2_test = test[['y2']]

In [263]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [264]:
xg_model1 = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=300, n_jobs=-1)

In [265]:
xg_model1.fit(x_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [266]:
y1_hat_train = xg_model1.predict(x_train)

In [267]:
accuracy_score(y1_train, y1_hat_train)

0.9743186582809225

In [268]:
xg_model2 = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=300, n_jobs=-1)

In [269]:
xg_model2.fit(x_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [270]:
y2_hat_train = xg_model2.predict(x_train)

In [271]:
accuracy_score(y2_train, y2_hat_train)

0.9700494791039141

In [197]:
y1_hat_test = xg_model1.predict(x_test)
y2_hat_test = xg_model2.predict(x_test)

In [198]:
accuracy_score(y2_test, y2_hat_test)

0.9594580105610208

In [199]:
accuracy_score(y1_test, y1_hat_test)

0.9672179483742998

# submission

In [272]:
submission = pd.read_csv('../sample_submission_2.csv')
req_uids = set(submission.user_id_hash)
dates = df.week_start.dt.date.unique()
dates.sort()

In [273]:
dates

array([datetime.date(2018, 10, 1), datetime.date(2018, 10, 8),
       datetime.date(2018, 10, 15), datetime.date(2018, 10, 22),
       datetime.date(2018, 10, 29), datetime.date(2018, 11, 5),
       datetime.date(2018, 11, 12), datetime.date(2018, 11, 19),
       datetime.date(2018, 11, 26), datetime.date(2018, 12, 3),
       datetime.date(2018, 12, 10)], dtype=object)

In [274]:
df.city = df.city.cat.codes
df.country = df.country.cat.codes
df.region = df.region.cat.codes
df.os_name = df.os_name.cat.codes
df.locale = df.locale.cat.codes
df.is_user_first_session = df.is_user_first_session.astype(np.int)
df['y1'] = df['y1'].astype(np.int)
df['y2'] = df['y2'].astype(np.int)

In [275]:
req_uids = set(submission.user_id_hash)
run_ids = req_uids
predict_df = pd.DataFrame(columns=['user_id_hash',
                                   'user_purchase_binary_7_days',
                                   'user_purchase_binary_14_days'])

for dts in reversed(dates[:-4]):
    t = df.loc[(df.user_id_hash.isin(run_ids)) &\
                     (df.week_start == dts), feature_cols].values
    cur_uids = df.loc[(df.user_id_hash.isin(run_ids)) &\
                     (df.week_start == dts), ['user_id_hash']].values

    pp1 = xg_model1.predict_proba(t)
    pp2 = xg_model1.predict_proba(t)

    print(f'{dts}: {pp1[:, 1].shape}')
    temp = pd.DataFrame(data={'user_id_hash': cur_uids.reshape(-1),
                              'user_purchase_binary_7_days': pp1[:,1],
                              'user_purchase_binary_14_days': pp2[:,1]})
    predict_df = pd.concat([predict_df, temp])
    run_ids = run_ids - set(cur_uids.reshape(-1))
    print(len(list(run_ids)))

2018-11-12: (60272,)
252296
2018-11-05: (38866,)
213430
2018-10-29: (36634,)
176796
2018-10-22: (34592,)
142204
2018-10-15: (32697,)
109507
2018-10-08: (28160,)
81347
2018-10-01: (22252,)
59095


In [276]:
predict_df.drop_duplicates(inplace=True)

In [277]:
left_out = list(set(submission.user_id_hash.unique()) -\
                set(predict_df.user_id_hash.unique()))

In [278]:
predict_df.user_purchase_binary_7_days.median()

0.0004101187514606863

In [279]:
predict_df.user_purchase_binary_14_days.median()

0.0002756034373305738

In [280]:
temp = pd.DataFrame(data={'user_id_hash': left_out,
                          'user_purchase_binary_7_days': 0.0004 * np.ones(len(left_out)),
                          'user_purchase_binary_14_days': 0.0003 * np.ones(len(left_out))})

In [281]:
predict_df = pd.concat([predict_df, temp])

In [282]:
predict_df.to_csv('../5_sub.csv', index=False, float_format='%.3f')